
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Securing Data in Unity Catalog

In this lab, you will create two objects secure tables:

1. Row Filtering and Column Masking Tables

2. Dynamic Views


### Learning Objectives
By the end of this demo, you will be able to:
1. Assign access permissions to the newly created view for account users and execute queries to validate the view's functionality and data integrity.
1. Develop a dynamic view of the cloned table, applying data redaction and access restrictions to enhance data security.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your specific catalog and the schema to the schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG <your catalog>;
USE SCHEMA <your catalog>.<schema>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-5

Run the code below and confirm that your current catalog is set to your unique catalog name and that the current schema is **default**.


In [0]:
SELECT current_catalog(), current_schema()

## B. Protect Columns and Rows with Column Masking and Row Filtering

### REQUIRED: Create the Table Customers
1. Run the code below to create the **customers** table in your **default** schema.

In [0]:
CREATE OR REPLACE TABLE customers AS
SELECT *
FROM samples.tpch.customer;

2. Run a query to view *10* rows from the **customers** table in your **default** schema. Notice that the table contains information such as **c_name**, **c_phone**, and **c_mktsegment**.

In [0]:
<FILL_IN>

### B1. Create a Function to Perform Column Masking
View the [Filter sensitive table data using row filters and column masks](https://docs.databricks.com/en/tables/row-and-column-filters.html) documentation for additional help.
1. Create a function named **phone_mask** that redacts the **c_phone** column in the **customers** table if the user is not a member of the ('admin') group using the `is_account_group_member` function. The **phone_mask** function should return the string *REDACTED PHONE NUMBER* if the user is not a member.

In [0]:
<FILL_IN>

2. Apply the column masking function **phone_mask** to the **customers** table using the `ALTER TABLE` statement.

In [0]:
<FILL_IN>

3. Run the cell below to confirm that you have correctly masked the **c_phone** column in the **customers** table. If an error is returned make sure the function is created and applied correctly.

In [0]:
%python
sdf = spark.sql('SELECT c_phone FROM customers GROUP BY c_phone')
value = sdf.collect()[0][0]

assert value == 'REDACTED PHONE NUMBER', 'The c_phone column should only contain the value "REDACTED PHONE NUMBER". Please create the correct column mask function and apply it to the customers table.'
print('You correctly applied the column mask to the customers table.')

4. Run the query below to view the **customers** table with the column mask applied. Confirm that the **c_phone** column displays the value *REDACTED PHONE NUMBER*.



In [0]:
SELECT *
FROM customers
LIMIT 10;

### B2. Create a Function to Perform Row Filtering
View the [Filter sensitive table data using row filters and column masks](https://docs.databricks.com/en/tables/row-and-column-filters.html) documentation for additional help.

1. Run the cell below to count the total number of rows in the **customers** table. Confirm that the table contains 750,000 rows of data.


In [0]:
SELECT count(*) AS TotalRows
FROM customers;

2. Create a function named **nation_filter** that filters on **c_nationkey** in the **customers** table if the user is not a member of the ('admin') group using the `is_account_group_member` function. The function should only return rows where **c_nationkey** equals *21*.

    View the [if function](https://docs.databricks.com/en/sql/language-manual/functions/if.html) documentation for additional help.

In [0]:
<FILL_IN>

3. Apply the function row filtering function `nation_filter` to the **customers** table using the `ALTER TABLE` statement.

In [0]:
<FILL_IN>

4. Run the cell below to confirm that you added row filtering to the **customers** table column **c_nationkey** correctly.


In [0]:
%python
sdf = spark.sql('SELECT c_nationkey FROM customers GROUP BY c_nationkey')
value = sdf.collect()[0][0]

assert value == 21, 'The c_nationkey column should only contain the value 21. Please create the correct row filter function and apply it to the customers table.'
print('You correctly applied the row filter.')

5. Run the query below to count the number of rows in the **customers** table for you since you've filtered out rows for users not admins. Confirm you can only view *29,859* rows (*where c_nationkey = 21*).

In [0]:
SELECT count(*) AS TotalRows
FROM customers;

5. Run the query below to view the **customers** table. 

Confirm the final table:
- redactes the **c_phone** column and
- filters rows based on the **c_nationkey** column for users who are not *admins*.

In [0]:
SELECT *
FROM customers;

## C. Protecting Columns and Rows with Dynamic Views


### REQUIRED: Create the Table Customers_new
1. Run the code below to create the **customers_new** table in your **default** schema.

In [0]:
CREATE OR REPLACE TABLE customers_new AS
SELECT *
FROM samples.tpch.customer;

2. Run a query to view *10* rows from the **customers_new** table in your **default** schema. Notice that the table contains information such as **c_name**, **c_phone**, and **c_mktsegment**.

In [0]:
SELECT *
FROM customers_new
LIMIT 10;

### C1. Create the Dynamic View
Let's create a view named **vw_customers** that presents a processed view of the **customers_new** table data with the following transformations:
1. Selects all columns from the **customers_new** table.

2. Redact all values in the **c_phone** column to *REDACTED PHONE NUMBER* unless you are in the `is_account_group_member('admins')`
    - HINT: Use a `CASE WHEN` statement in the `SELECT` clause.

3. Restrict the rows where **c_nationkey** is equal to *21* unless you are in the `is_account_group_member('admins')`.
    - HINT: Use a `CASE WHEN` statement in the `WHERE` clause.

In [0]:
<FILL_IN>

3. Run the cell below to confirm that you have correctly masked the **c_phone** column in the **vw_customers** view. If an error is returned make sure the function is created and applied correctly.

In [0]:
%python
sdf = spark.sql('SELECT c_phone FROM vw_customers GROUP BY c_phone')
value = sdf.collect()[0][0]

assert value == 'REDACTED PHONE NUMBER', 'The c_phone column should only contain the value "REDACTED PHONE NUMBER". Please use the following expression in the SELECT clause: CASE WHEN is_account_group_member("admins") THEN c_phone ELSE "REDACTED PHONE NUMBER" END as c_phone.'
print('You correctly applied the column mask to the vw_customers view.')

4. Run the cell below to confirm that you added row filtering to the **customers** table column **c_nationkey** correctly.

In [0]:
%python
sdf = spark.sql('SELECT c_nationkey FROM vw_customers GROUP BY c_nationkey')
value = sdf.collect()[0][0]

assert value == 21, 'The c_nationkey column should only contain the value 21. Please create the correct row filter in the view using the following expression in the WHERE clause: CASE WHEN is_account_group_member("admins") THEN TRUE ELSE c_nationkey = 21 END.'
print('You correctly applied the row filter to the view.')

5. Display the data in the **vw_customers** view. Confirm the **c_phone** column is redacted.

In [0]:
SELECT * 
FROM vw_customers;

6. Count the number of rows in the **vw_customers** view. Confirm the view contains *29,859* rows.

In [0]:
SELECT count(*)
FROM vw_customers;

### C2. Issue Grant Access to View
1. Let us issue a grant for "account users" to view the **vw_customers** view.

**NOTE:** You will also need to provide users access to the catalog and schema. In this shared training environment, you are unable to grant access to your catalog to other users.


In [0]:
<FILL_IN>

2. Use the `SHOW` statement to displays all privileges (inherited, denied, and granted) that affect the **vw_customers** view. Confirm that the **Principal** column contains *account users*.

View the [SHOW GRANTS](https://docs.databricks.com/en/sql/language-manual/security-show-grant.html) documentation for help.

In [0]:
<FILL_IN>


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>